In [1]:
!sudo apt install tesseract-ocr
!sudo apt-get install poppler-utils
!pip install pytesseract pdf2image zero-shot-re neo4j

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,128 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

### reading pdf document with OCR


In [1]:
# using pytesseract library to extract text from pdf
import requests
import pdf2image
import pytesseract

In [2]:
pdf = requests.get('https://arxiv.org/pdf/2110.03526.pdf')
doc = pdf2image.convert_from_bytes(pdf.content)

In [5]:
# get the article text
article = []
for page_number, page_data in enumerate(doc):
  txt = pytesseract.image_to_string(page_data).encode("utf-8")
  if page_number<6:
    article.append(txt.decode("utf-8"))

In [6]:
article_text = " ".join(article)

In [7]:
article_text

'Mohammadreza Ahmadi\n\nTissue Engineering and Regeneration of Skin\nand Hair Follicle Growth From Stem Cells\n\nINTRODUCTION\n\nMany people with skin diseases such as chronic wounds, non-healing and diabetic\nulcers need reconstruction and regeneration of their skin. In addition, the medical industry also\nneeded a method of skin rejuvenation and reconstruction for cosmetic purposes, even for\nhealthy people. Reconstructive medicine used the method to deliver pluripotent stem cells to the\ntargeted tissue.\n\n33 years after the introduction of bone marrow stem cells, fat-derived stem cells have\nbecome an excellent source for cell therapy. In 1961, two Canadian scientists first introduced\nstem cells. These cells, later found to be hematopoietic stem cells, have been used successfully\nto treat leukemia and some severe autoimmune diseases called bone marrow transplants. In\n1968, another stem cell was introduced into the bone marrow, which has been shown to be\neffective due to its hi

In [8]:

import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
"\n".join(["Ahmadi\n\nTissue  Engineering and Regeneration of Skin\nand Hair Follicle.Cells\n\nINTRODUCTION\n\nMany people with ski"])

'Ahmadi\n\nTissue  Engineering and Regeneration of Skin\nand Hair Follicle.Cells\n\nINTRODUCTION\n\nMany people with ski'

In [10]:
def clean_text(text):
  """remove section titles and figure description from the text"""
  clean = "\n".join([row for row in text.split("\n") if (len(row.split(" ")))>3 and not (row.startswith("(a)")) and not row.startswith("Figure")])
  return clean
text = article_text.split("INTRODUCTION")[1]
ctext = clean_text(text)
sentence = nltk.tokenize.sent_tokenize(ctext)

In [11]:
sentence

['Many people with skin diseases such as chronic wounds, non-healing and diabetic\nulcers need reconstruction and regeneration of their skin.',
 'In addition, the medical industry also\nneeded a method of skin rejuvenation and reconstruction for cosmetic purposes, even for\nhealthy people.',
 'Reconstructive medicine used the method to deliver pluripotent stem cells to the\n33 years after the introduction of bone marrow stem cells, fat-derived stem cells have\nbecome an excellent source for cell therapy.',
 'In 1961, two Canadian scientists first introduced\nstem cells.',
 'These cells, later found to be hematopoietic stem cells, have been used successfully\nto treat leukemia and some severe autoimmune diseases called bone marrow transplants.',
 'In\n1968, another stem cell was introduced into the bone marrow, which has been shown to be\neffective due to its high ability to regulate immunity in many diseases, including skin, bone, joint\ndiseases, heart, brain, nerves, and kidney.',
 '

In [12]:
import hashlib
def query_raw(text,url ="https://bern.korea.ac.kr/plain"):
  """biomedical entity linking API"""
  return requests.post(url, data={"sample_text":text}).json()

In [13]:
entity_list = []
# the last sentence is invalid
for s in sentence[:-1]:
  entity_list.append(query_raw(s))
entity_list

[{'denotations': [{'id': ['MESH:D012871', 'BERN:254048701'],
    'obj': 'disease',
    'span': {'begin': 17, 'end': 30}},
   {'id': ['BERN:259009101'],
    'obj': 'disease',
    'span': {'begin': 39, 'end': 53}},
   {'id': ['MESH:D014456', 'BERN:255105301'],
    'obj': 'disease',
    'span': {'begin': 71, 'end': 86}}],
  'logits': {'disease': [[{'end': 30,
      'id': 'MESH:D012871\tBERN:254048701',
      'start': 17},
     0.9999999403953552],
    [{'end': 53, 'id': 'BERN:259009101', 'start': 39}, 0.9999999403953552],
    [{'end': 86, 'id': 'MESH:D014456\tBERN:255105301', 'start': 71},
     0.9999999403953552]],
   'drug': [],
   'gene': [],
   'species': [[{'end': 11, 'start': 5}, 0.9999998807907104]]},
  'project': 'BERN',
  'sourcedb': '',
  'sourceid': '8081d17f50e197193712d4281e8e4fbb5e49d76b457a95aa20f9216f-Thread-6647173',
  'text': 'Many people with skin diseases such as chronic wounds, non-healing and diabetic ulcers need reconstruction and regeneration of their skin.',
  'ti

In [15]:
parsed_entities = []
for entities in entity_list:
  e = []
  # if there are not entities in the text
  if not entities.get("denotations"):
    parsed_entities.append({"text":entities["text"],"text_sha256":hashlib.sha256(entities["text"].encode("utf-8")).hexdigest()})
    continue
  for entity in entities["denotations"]:
    other_ids = [id for id in entity["id"] if not id.startswith("BERN")]
    entity_type = entity["obj"]
    entity_name = entities["text"][entity["span"]["begin"]:entity["span"]["end"]]
    try:
      entity_id =[ id for id in entity["id"] if id.startswith("BERN")][0]
    except IndexError:
      entity_id = entity_name
    e.append({"entity_id":entity_id,"other_ids":other_ids,"entity_type":entity_type,"entity_name":entity_name})
  parsed_entities.append({"entities":e,"text":entities["text"],"text_sha256":hashlib.sha256(entities["text"].encode("utf-8")).hexdigest()})


In [16]:
parsed_entities

[{'entities': [{'entity_id': 'BERN:254048701',
    'entity_name': 'skin diseases',
    'entity_type': 'disease',
    'other_ids': ['MESH:D012871']},
   {'entity_id': 'BERN:259009101',
    'entity_name': 'chronic wounds',
    'entity_type': 'disease',
    'other_ids': []},
   {'entity_id': 'BERN:255105301',
    'entity_name': 'diabetic ulcers',
    'entity_type': 'disease',
    'other_ids': ['MESH:D014456']}],
  'text': 'Many people with skin diseases such as chronic wounds, non-healing and diabetic ulcers need reconstruction and regeneration of their skin.',
  'text_sha256': '95be1713dbb23e86525959e49ea5a196b49f3bbc1ce8c6cbfa373f176c9d39ae'},
 {'text': 'In addition, the medical industry also needed a method of skin rejuvenation and reconstruction for cosmetic purposes, even for healthy people.',
  'text_sha256': '3424add78e6e71cf20906f1fc0bd050ace5cd3d9f71a430c2f8535dd21354f66'},
 {'text': 'Reconstructive medicine used the method to deliver pluripotent stem cells to the 33 years after 

In [17]:
from neo4j import GraphDatabase
import pandas as pd

host = "bolt://54.165.120.59:7687"
user = "neo4j"
password = "farm-fog-plugs"
driver = GraphDatabase.driver(host,auth =(user,password))

def neo4j_query(query, params =None):
  with driver.session() as session:
    result =session.run(query,params)
    return pd.DataFrame([r.values() for r in result],columns=result.keys())

In [18]:
# importing article and author into the graph.the article node contain only title
author = article_text.split("\n")[0]
title = " ".join(article_text.split("\n")[2:4])

In [19]:
neo4j_query("""
MERGE(a:Author{name:$author})
MERGE(b:Article{title:$title})
MERGE (a)-[:WROTE]->(b)
""",{"title":title,"author":author})

""


In [20]:
neo4j_query("""
MATCH (a:Article)
UNWIND $data as row
MERGE(s:Sentence{id:row.text_sha256})
SET s.text = row.text
MERGE (a)-[:HAS_SENTENCE]->(s)
WITH s,row.entities as entities
UNWIND entities as entity
MERGE (e:Entity{id:entity.entity_id})
ON CREATE SET e.other_ids = entity.other_ids,
e.name = entity.entity,
e.type = entity.entity_type
MERGE (s)-[m:MENTIONS]->(e)
ON CREATE SET m.count = 1
ON MATCH SET m.count = m.count+1
""",{"data":parsed_entities})

""


### SEARCH ENGINE

In [24]:
neo4j_query("""
MATCH (e:Entity)<-[:MENTIONS]-(s:Sentence)
WHERE e.name = "autoimmune diseases"
RETURN s.text as result
""")

,result


### Co-occurrence analysis

In [27]:
neo4j_query("""
MATCH (e1:Entity)<-[:MENTIONS]-()-[:MENTIONS]->(e2:Entity)
WHERE id(e1) < id(e2)
RETURN e1.name as entity1, e2.name as entity2, count(*) as cooccurrence
ORDER BY cooccurrence
DESC LIMIT 3
""")

,entity1,entity2,cooccurrence
0,None,None,48


In [29]:
neo4j_query("""
MATCH (a:Author)- [:WROTE]->()-[:HAS_SENTENCE]->()-[:MENTIONS]->(e:Entity)
RETURN a.name as author , e.name as entity, count(*) as count
ORDER BY count DESC
LIMIT 5
""")

,author,entity,count
0,Mohammadreza Ahmadi,None,57


## Relation extraction

In [30]:
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

In [32]:
model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
relations = ["associated","interacts"]
extractor = RelationExtractor(model,tokenizer,relations)

DEBUG:filelock:Attempting to acquire lock 140299747587344 on /root/.cache/huggingface/transformers/067f38dffc6c575970ebe31a13476b0f1b1059010773d6c930cb93bb3bdba91f.6b2ed003371c5ec62febae1f3cf849d3ffff6dbfec976724bdad0a0c6e0d38b0.lock
DEBUG:filelock:Lock 140299747587344 acquired on /root/.cache/huggingface/transformers/067f38dffc6c575970ebe31a13476b0f1b1059010773d6c930cb93bb3bdba91f.6b2ed003371c5ec62febae1f3cf849d3ffff6dbfec976724bdad0a0c6e0d38b0.lock


Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140299747587344 on /root/.cache/huggingface/transformers/067f38dffc6c575970ebe31a13476b0f1b1059010773d6c930cb93bb3bdba91f.6b2ed003371c5ec62febae1f3cf849d3ffff6dbfec976724bdad0a0c6e0d38b0.lock
DEBUG:filelock:Lock 140299747587344 released on /root/.cache/huggingface/transformers/067f38dffc6c575970ebe31a13476b0f1b1059010773d6c930cb93bb3bdba91f.6b2ed003371c5ec62febae1f3cf849d3ffff6dbfec976724bdad0a0c6e0d38b0.lock
DEBUG:filelock:Attempting to acquire lock 140298543915472 on /root/.cache/huggingface/transformers/b9f8d92aa5a32cfe504c3524c173dc611dbe81d49392f40601286b94ee1e1169.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 140298543915472 acquired on /root/.cache/huggingface/transformers/b9f8d92aa5a32cfe504c3524c173dc611dbe81d49392f40601286b94ee1e1169.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140298543915472 on /root/.cache/huggingface/transformers/b9f8d92aa5a32cfe504c3524c173dc611dbe81d49392f40601286b94ee1e1169.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 140298543915472 released on /root/.cache/huggingface/transformers/b9f8d92aa5a32cfe504c3524c173dc611dbe81d49392f40601286b94ee1e1169.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Attempting to acquire lock 140298543850192 on /root/.cache/huggingface/transformers/68e5260dea718cdc2daf27dc106fd8741636b03e3173b5492e57a7fa525ca33b.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140298543850192 acquired on /root/.cache/huggingface/transformers/68e5260dea718cdc2daf27dc106fd8741636b03e3173b5492e57a7fa525ca33b.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140298543850192 on /root/.cache/huggingface/transformers/68e5260dea718cdc2daf27dc106fd8741636b03e3173b5492e57a7fa525ca33b.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140298543850192 released on /root/.cache/huggingface/transformers/68e5260dea718cdc2daf27dc106fd8741636b03e3173b5492e57a7fa525ca33b.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Attempting to acquire lock 140298543708880 on /root/.cache/huggingface/transformers/9b7535fe1c0da28aa7cc66b7f34529d984f535c401be8352f6adeb25f7870def.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 140298543708880 acquired on /root/.cache/huggingface/transformers/9b7535fe1c0da28aa7cc66b7f34529d984f535c401be8352f6adeb25f7870def.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140298543708880 on /root/.cache/huggingface/transformers/9b7535fe1c0da28aa7cc66b7f34529d984f535c401be8352f6adeb25f7870def.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 140298543708880 released on /root/.cache/huggingface/transformers/9b7535fe1c0da28aa7cc66b7f34529d984f535c401be8352f6adeb25f7870def.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


In [33]:
import itertools
#candidate sentence where there is more than single entity pesent
candidates = [s for s in parsed_entities if (s.get("entities")) and (len(s["entities"])>1)]

In [37]:
candidates

[{'entities': [{'entity_id': 'BERN:254048701',
    'entity_name': 'skin diseases',
    'entity_type': 'disease',
    'other_ids': ['MESH:D012871']},
   {'entity_id': 'BERN:259009101',
    'entity_name': 'chronic wounds',
    'entity_type': 'disease',
    'other_ids': []},
   {'entity_id': 'BERN:255105301',
    'entity_name': 'diabetic ulcers',
    'entity_type': 'disease',
    'other_ids': ['MESH:D014456']}],
  'text': 'Many people with skin diseases such as chronic wounds, non-healing and diabetic ulcers need reconstruction and regeneration of their skin.',
  'text_sha256': '95be1713dbb23e86525959e49ea5a196b49f3bbc1ce8c6cbfa373f176c9d39ae'},
 {'entities': [{'entity_id': 'BERN:106966401',
    'entity_name': 'leukemia',
    'entity_type': 'disease',
    'other_ids': ['MESH:D007938']},
   {'entity_id': 'BERN:259931301',
    'entity_name': 'autoimmune diseases',
    'entity_type': 'disease',
    'other_ids': ['MESH:D001327', 'OMIM:109100']}],
  'text': 'These cells, later found to be hema

In [38]:
predicted_rels = []
for c in candidates:
  combinations = itertools.combinations([{"name": x['entity_name'], "id":x["entity_id"]} for x in c["entities"]],2)
  for combination in list(combinations):
    try:
      ranked_rels = extractor.rank(text =c["text"].replace(","," "),head = combination[0]["name"],tail=combination[1]["name"])
      #define threshold for the most probable relation
      if ranked_rels[0][1] > 0.85:
        predicted_rels.append({"head":combination[0]["id"],"tail":combination[1]["id"], "type":ranked_rels[0][0], "source":c["text_sha256"]})
    except:
      pass

In [39]:
neo4j_query("""
UNWIND $data as row
MATCH (source:Entity{id:row.head})
MATCH (Target:Entity{id:row.tail})
MATCH (text:Sentence{id:row.source})
MERGE (source)-[:REL]->(r:Relation{type:row.type})-[:REL]->(target)
MERGE (text)-[:MENTIONS]->(r)
""",{"data":predicted_rels})

""


In [41]:
# extracted relationship between entities and the source text
neo4j_query("""
MATCH (s:Entity)-[:REL]->(r:Relation)-[:REL]->(t:Entity),
(r)<-[:MENTIONS]-(st:Sentence)
RETURN s.name as source_entity, t.nme as target_entity, r.type as type, st.text as source_text
""")

,source_entity,target_entity,type,source_text
0,None,None,associated,Many people with skin diseases such as chronic...
1,None,None,associated,Many people with skin diseases such as chronic...
2,None,None,associated,"These cells, later found to be hematopoietic s..."
3,None,None,interacts,"Furthermore, the primary sources of extracellu..."


## External database enrichment

In [43]:
# mesh enrichment
neo4j_query("""
MATCH (e:Entity)
WHERE e.name ="Epidermolysis bullosa"
WITH e,
[id in e.other_ids WHERE id contains "MESH" | split(id,":")[1]][0] as meshId
CALL apoc.load.json("https://id.nlm.nih.gov/mesh/lookup/details?descriptor=" + meshId)
YIELD value
RETURN value
""")

,value
